# Process Stations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pkg_resources import resource_filename

from climpyrical.gridding import scale_model_obs
from climpyrical.data import read_data
from climpyrical.cmd.find_matched_model_vals import add_model_values

### Load station data for processing

### Parameters are loaded from config_example.yml
Example configuration from config_example.yml:
```
    paths:
        preprocessed_model_path: /data/results/intermediate/preprocessed_netcdf/
    RL50:
        station_dv: "RL50 (kPa)"
        station_path: 'data/station_inputs/Interim_snow_rain_load_LR_composite_stations_tbd_v4.csv'
        input_model_path: 'data/model_inputs/snw_rain_CanRCM4-LE_ens35_1951-2016_max_rl50_load_ensmean.nc'
        medians: 
            value: 0.4
            action: "multiply"
        fill_glaciers: True
```

In [2]:
# Parameters
name = "RAnn"
station_dv = "annual_rain (mm)"
station_path = "data/station_inputs/rain_annual_mean_doy_MSC_25yr_for_maps.csv"
input_model_path = (
    "data/model_inputs/rain_CanRCM4-LE_ens35_1951-2016_ann_sum_ensmean.nc"
)
medians = {"value": 726, "action": "multiply"}
fill_glaciers = True
output_notebook_path = "/data/results/intermediate/notebooks/"
preprocessed_model_path = "/data/results/intermediate/preprocessed_netcdf/"
preprocessed_stations_path = "/data/results/intermediate/preprocessed_stations/"
output_reconstruction_path = "/data/results/netcdf/"
output_tables_path = "/data/results/TableC2/"
output_figure_path = "/data/results/figures/"
mask_path = "data/masks/canada_mask_rp.nc"
north_mask_path = "data/masks/canada_mask_north_rp.nc"
nbcc_loc_path = "data/station_inputs/NBCC_2020_new_coords.xlsm"


In [3]:
# Parameters cell

In [4]:
df = pd.read_csv(resource_filename("climpyrical", station_path), index_col=None)

Check the columns for standard names, and standardize them.

This process will not catch any and all possible inputs, so refer to documentation for 
expected column names

In [5]:
def check_df_columns(df):
    if 'longitude' in df.columns:
        df=df.rename(columns={'longitude': 'lon'})
    if 'long' in df.columns:
        df=df.rename(columns={'long': 'lon'})
    if 'latitude' in df.columns:
        df=df.rename(columns={'latitude': 'lat'})
    if 'name' in df.columns:
        df=df.rename(columns={'name': 'station_name'})
    if 'Name' in df.columns:
        df=df.rename(columns={'Name': 'station_name'})
    if "prov" in df.columns:
        df=df.rename(columns={"prov": "province"})
    if "elev" in df.columns:
        df=df.rename(columns={"elev": "elev (m)"})
    if "elevation (m)" in df.columns:
        df=df.rename(columns={"elevation (m)": "elev (m)"})
    return df

df = check_df_columns(df)
df.head(5)

if np.any(np.isnan(df[["lon", "lat", "elev (m)", station_dv]].values)):
    raise ValueError("NaN detected in station input file.")

Load preprocessed model at target resolution

In [6]:
ds = read_data(resource_filename("climpyrical", f"{preprocessed_model_path}{name}.nc"))
(dv, ) = ds.data_vars

Perform unit conversion if necessary

Exclude suspect RL50 values with true 0.0

In [7]:
# calculate ratios with applied correction
# Note the ratios are calculated in K for Temperature fields
if ds[dv].attrs["units"] == "degC" and "degC" in station_dv:
    print("Temperature DV detected in degC. Converitng to Kelvin")
    K = 273.15 # K
    df[station_dv] += K
    ds[dv] += K

if 'RL50 (kPa)' in station_dv:
    df = df[df[station_dv] != 0.0]

Use matching script to find matching model values

In [8]:
df = add_model_values(ds=ds, df=df)
df.head(5)

INFO:root:Detect units: mm


INFO:root:rlat or rlon not detected in input file.converting assumes WGS84 coords to rotated pole


INFO:root:Matching coordinates now


INFO:root:Locating corresponding model valuesInterpolating to nearest if matched model value is NaN


,history_id,station_name,province,lon,lat,elev (m),annual_rain (mm),rlat,rlon,irlat,irlon,model_values
0,12,CENTRAL SAANICH VEYANESS,BC,-123.417,48.5833,53,811,4.004513,-17.159686,745,387,1436.924438
1,13,CENTRAL SAANICH ISL VIEW,BC,-123.392,48.5733,38,815,3.989632,-17.147322,745,387,1436.924438
2,15,CHEMAINUS,BC,-123.742,48.9350,75,1264,4.407283,-17.245771,754,385,1843.940918
3,24,COWICHAN BAY CHERRY POINT,BC,-123.557,48.7111,1,945,4.155661,-17.204997,748,386,1566.150879
4,26,COWICHAN LAKE FORESTRY,BC,-124.133,48.8244,177,2059,4.389211,-17.525927,754,378,1953.911865


Group stations that land in the same index in rlat and rlon (land in the same grid cell)

This means that they are in the same grid cell and need to be aggregated

In [9]:
agg_dict = {
    station_dv: 'mean', 
    'rlat':'mean', 
    'rlon':'mean', 
    'lat': 'mean', 
    'lon': 'mean', 
    'elev (m)': 'mean',
    'station_name': 'first',
    'province': 'first',
    "model_values": "mean"
}

# Province key is used for WP10 and WP50 for
# special treatment of Atlantic/Far NW areas
if "province" not in df.columns:
    agg_dict = agg_dict.pop("province")

df_match = df.groupby(
    ['irlat', 'irlon'], as_index=False
).agg(
    agg_dict
)


irlat = df_match.irlat
irlon = df_match.irlon

In [10]:
df_match

,irlat,irlon,annual_rain (mm),rlat,rlon,lat,lon,elev (m),station_name,province,model_values
0,552,1011,793.0,-4.517790,10.457750,42.0333,-82.9000,191.0,HARROW CDA,ON,884.890747
1,553,1007,850.0,-4.474659,10.302673,42.1034,-83.0945,182.0,AMHERSTBURG,ON,892.609375
2,553,1015,773.0,-4.477661,10.621608,42.0431,-82.6739,200.0,KINGSVILLE MOE,ON,881.386108
3,558,1009,814.0,-4.286849,10.373390,42.2756,-82.9556,190.0,WINDSOR A,ON,882.435547
4,563,1021,792.0,-4.073695,10.891633,42.3900,-82.2153,180.0,CHATHAM WPCP,ON,843.279114
...,...,...,...,...,...,...,...,...,...,...,...
1363,1252,425,84.0,26.462377,-15.465896,69.5833,-140.1830,7.0,KOMAKUK BEACH A,YT,159.541245
1364,1269,788,55.0,27.224763,0.601810,74.7169,-94.9694,68.0,RESOLUTE CARS,NU,126.862511
1365,1322,640,27.0,29.596811,-5.969242,76.2333,-119.3330,12.0,MOULD BAY A,NT,104.951614
1366,1391,826,28.0,32.645419,2.272431,79.9833,-85.9333,10.0,EUREKA A,NU,116.047218


Verify that they are correlated

Text(0, 0.5, 'Model')

In [11]:
plt.scatter(df_match[station_dv], df_match.model_values)
plt.title("Correlation between station and models")
plt.xlabel('Stations')
plt.ylabel('Model')

Now each station should be matched with a value grid cell value

### Normalize the model mean to match that of the station distribution
Find a factor, $\beta$ such that
$$\mu_s - \frac{\mu_m}{\beta} \approx 0$$ where $\mu_s$ is the spatial station mean and $\mu_m$ is the spatial model mean.

$\beta$ is simply:

$$\beta = \frac{\sum_i^{N_s}{M_i}}{\sum_i^{N_s}{S_i}}$$

In [12]:
ratio, best_tol = scale_model_obs(df_match.model_values, df_match[station_dv])
assert not np.any(np.isnan(ratio))
df_match = df_match.assign(ratio=ratio)
print("Scaling factor:", best_tol)

Scaling factor: 1.1600179694085848


In [13]:
df_match.head(3)

,irlat,irlon,annual_rain (mm),rlat,rlon,lat,lon,elev (m),station_name,province,model_values,ratio
0,552,1011,793.0,-4.517790,10.457750,42.0333,-82.9000,191.0,HARROW CDA,ON,884.890747,1.039557
1,553,1007,850.0,-4.474659,10.302673,42.1034,-83.0945,182.0,AMHERSTBURG,ON,892.609375,1.104644
2,553,1015,773.0,-4.477661,10.621608,42.0431,-82.6739,200.0,KINGSVILLE MOE,ON,881.386108,1.017368


In [14]:
# convert back to degC
if ds[dv].attrs["units"] == "degC" and "degC" in station_dv:
    print("Temperature DV detected in degC. Converitng to Kelvin")
    K = 273.15 # K
    df_match[station_dv] -= K
    df_match.model_values -= K

In [15]:
df_match.to_csv(
    resource_filename(
        "climpyrical",
        f"{preprocessed_stations_path}{name}.csv"
    ), index=False
)

In [16]:
assert np.all(df_match['ratio'] >= 0)